In [ ]:
#INFO - Librarite e perdorura 

import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# INFO - Leximi i dataset-it nga source!

dataset=pd.read_csv('../data/gjobat-e-leshuara.csv')

In [ ]:
# Para Procesimi

# Kontrollojme se a mungon ndonje vlere
dataset.isnull().sum()

# Ndryshimi i emrave te kolonave te caktuara
dataset.rename(columns={'LLOJI_KOMPAN': 'LLOJI_KOMPANISE', 'NR_TATIM': 'NR_TATIMIT'}, inplace=True)
print(dataset)

In [ ]:
# Definimi i tipeve

print(dataset.dtypes)

In [ ]:
# Paraqitja e numrit te rreshtave dhe kolonave

dataset.shape

In [ ]:
# Integrimi

dataset['MUAJI/VITI'] = dataset['MUAJI'].astype(str) + '/' + dataset['VITI'].astype(str)

In [ ]:
# Agregimi

Vlerat_bazuar_ne_vite = dataset.groupby('VITI')['VLERA'].sum()
print(Vlerat_bazuar_ne_vite)